***Current WIP***

Importing required libraries

In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import os
import requests
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import argparse
import csv
from typing import List
from requests_html import HTMLSession


Infinite Scrolling (WIP)

In [6]:
hashtag='lipstick'
browser = webdriver.Chrome('/usr/local/bin/chromedriver')
browser.get('https://www.instagram.com/explore/tags/'+hashtag)

links=[]
for i in range(0,100):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

source = browser.page_source
data= BeautifulSoup(source, 'html.parser')
body = data.find('body')
script = body.find('span')

for link in script.findAll('a'):
     if re.match("/p", link.get('href')):
        links.append('https://www.instagram.com'+link.get('href'))


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=75.0.3770.142)


In [523]:
##Testing
#len(links)
#print(links)

**Primary post scrapping**

In [7]:
likes = []
comments = []
t_hashtags= []
oglink= []
image_links=[]
user_links=[]
count=[]

##Secondary scrapper for usernames!

for i in range(len(links)):
    webpage_source = requests.get(str(links[i])).text
    page_content = BeautifulSoup(webpage_source, 'html.parser')
    #print(page_content)
    hashtags= []
    count.append(i)
    for tag in page_content.find_all("meta"):
        if tag.get("property", None) == "instapp:hashtags":
            hashtags.append(str("#" + tag.get("content", None)))
        elif tag.get("property", None) == "video:tag":
            hashtags.append(str("#" + tag.get("content", None)))
        elif tag.get("property", None) == "og:url":
            oglink.append(tag.get("content", None))
        elif tag.get("property", None) == "og:description":
            descrip = tag.get("content", None).split()
            #print(descrip)
            likes.append(descrip[0])
            comments.append(descrip[2])
        elif tag.get("property", None) == "og:image":
            image_links.append(tag.get("content", None))
    
    t_hashtags.append(hashtags)
    deets = page_content.find('link', attrs={'rel': 'canonical'})
    instauser = str(deets.get('href'))[:-14]
    user_links.append(instauser)

NameError: name 'username' is not defined

In [542]:
#print(user_links)
#test_links=['https://www.instagram.com/seduceme.pe/', 'https://www.instagram.com/p/Bz8a5z9DsBN/']

**Secondary scraping user**

In [8]:
username= []
followers= []
following= []
posts= []
r_userbio=[]
info={}

for i in range(len(user_links)):
    soup = BeautifulSoup(requests.get(str(user_links[i])).text, 'html.parser')
    data = soup.find('meta', attrs={'property': 'og:description'})
    if data == None:
        print(i)
        r_userbio.append('no description available')
        username.append('no info available')
        followers.append('no info available')
        following.append('no info available')
        posts.append('no info available')
        
    else:
        text = data.get('content').split()
        username.append(' '.join(soup.title.string.split()[:-5]))
        followers.append(text[0])
        following.append(text[2])
        posts.append(text[4])
        
        descript = soup.find('script', attrs={'type': 'application/ld+json'})
        if descript == None: 
            r_userbio.append("no description available")
        else:
            jscript = json.loads(descript.string)    
            r_userbio.append(jscript.get("description"))

userbio =[]
    
for i in range(len(r_userbio)):
    if r_userbio[i] == None:
        userbio.append("no description available")
    else:
        userbio.append(r_userbio[i].replace('\n', ''))
        
info["UserID"] = count
info["User"] = username
info["Followers"] = followers
info["Following"] = following
info["Posts"] = posts
info["Userbio"] = userbio

#print(likes[10], comments[10], t_hashtags[10], oglink[10], image_links[10], user_links)
scraped = {'PostID': count , 'Username': username,'OGlink': oglink, 'Likes': likes, "Comments": comments, "Hashtags": t_hashtags, 
           "Imglink": image_links }

df1 = pd.DataFrame(scraped)
df2 = pd.DataFrame(info)


NameError: name 'scraped' is not defined

In [545]:
df1[1::]

,Username,OGlink,Likes,Comments,Hashtags,Imglink
1,FASHION BEAUTY AND STYLE (@moda2528),https://www.instagram.com/p/Bz9PsIyJGAi/,"3,660",33,"[#beautyaddict, #instagram, #mua, #smokeyeyes,...",https://scontent-syd2-1.cdninstagram.com/vp/48...
2,@millydoesmakeup_,https://www.instagram.com/p/Bz8eoK5HnAo/,"6,530",87,"[#hudabeauty, #maccosmetics, #mua, #bretmanvan...",https://scontent-syd2-1.cdninstagram.com/vp/00...
3,FASHION BEAUTY AND STYLE (@moda2528),https://www.instagram.com/p/Bz8v-HUp5EM/,"3,689",29,"[#eyelook, #undiscoveredmuas, #glam, #lipstick...",https://scontent-syd2-1.cdninstagram.com/vp/19...
4,Da Glossiest Of Them All 💋 (@dymeblock),https://www.instagram.com/p/Bz9AqRgjhnL/,575,23,"[#blackgirlmagic, #lipglosslover, #makeupartis...",https://scontent-syd2-1.cdninstagram.com/vp/33...
5,Belrose Beauty (@belrosebeauty),https://www.instagram.com/p/Bz89XnNHI82/,11.4k,29,"[#belrosebeauty, #glowup, #glitter, #nyxcosmet...",https://scontent-syd2-1.cdninstagram.com/vp/b2...
6,Belrose Beauty (@belrosebeauty),https://www.instagram.com/p/Bz9elsinrCR/,"1,443",8,"[#makeuptutorials, #anastasiabeverlyhills, #ey...",https://scontent-syd2-1.cdninstagram.com/vp/ae...
7,beautybynena (@nena),https://www.instagram.com/p/Bz9HG88JB4e/,25.5k,394,"[#chicagoblogger, #beautyproducts, #abh, #eyes...",https://scontent-syd2-1.cdninstagram.com/vp/92...
8,OFRA Cosmetic Laboratories (@ofracosmetics),https://www.instagram.com/p/Bz89n0pFlD1/,"1,922",42,[#ofraxnikkietutorials],https://scontent-syd2-1.cdninstagram.com/vp/a3...
9,Michelle Lomeli🦇 (@mimiartistry),https://www.instagram.com/p/Bz9oGMupVN4/,140,12,[],https://scontent-syd2-1.cdninstagram.com/vp/96...
10,Order nội địa Nhật (@athena.beauties),https://www.instagram.com/p/Bz9onoVBBgF/,2,0,"[#cosmetic, #jillstuartlipstick, #jillstuartro...",https://scontent-syd2-1.cdninstagram.com/vp/ab...


In [522]:
df2[1::]

,User,Followers,Following,Posts,Userbio
1,FASHION BEAUTY AND STYLE (@moda2528),168.2k,"3,700","2,912",Beauty community ✨Owner: @gloria_medinap @make...
2,@millydoesmakeup_,71.5k,945,184,Milly14 | Makeup💄|🇬🇧| Aspiring MUA|📨business e...
3,FASHION BEAUTY AND STYLE (@moda2528),168.2k,"3,700","2,912",Beauty community ✨Owner: @gloria_medinap @make...
4,Da Glossiest Of Them All 💋 (@dymeblock),"5,898",76,98,•Beauty &amp; Skincare Head Huncho •Use Code “...
5,Belrose Beauty (@belrosebeauty),915.2k,577,"3,245","The BEST of makeup, hair, &amp; new releases! ..."
6,Belrose Beauty (@belrosebeauty),915.2k,577,"3,245","The BEST of makeup, hair, &amp; new releases! ..."
7,beautybynena (@nena),363.5k,455,"1,035",🍳ʟᴀᴛɪɴᴀ ɪɴfʟᴜᴇɴᴄᴇʀ/ʙᴇᴀᴜᴛʏ &amp; ʟɪfᴇsᴛʏʟᴇ ᴠʟᴏɢ...
8,OFRA Cosmetic Laboratories (@ofracosmetics),1.4m,407,"5,589",✨Hey Beauties!🇺🇸Made in USA🌴OFRA HQ in Sunny F...
9,Michelle Lomeli🦇 (@mimiartistry),16.1k,421,"1,851","More Art, Less Hate 🤘🏾xɪᴄᴀɴᴀ | @shopmimiartist..."
10,Order nội địa Nhật (@athena.beauties),17,10,7,"Jill Stuart, hàng nội địa nhật ⛩️ Order, Có Sẵ..."


In [536]:
import wget
print('Beginning file download with wget')

for i in range(len(image_links)):
    url = str(image_links[i])
    wget.download(url, str('/Users/ahuang17/Desktop/Photos/PostID' + str(i) + '.jpg')) 


Beginning file download with wget


In [549]:
##Logged in version

uploadDate=[]
##author = []
##cp_text = []
#user_count= []
com_count = []
##c_user = []
##c_ulink = []

class InstagramBot():
    def __init__(self, email, password):
        self.browser = webdriver.Chrome()
        self.email = email
        self.password = password
    
    def signIn(self):
        self.browser.get('https://www.instagram.com/accounts/login/')

        emailInput = self.browser.find_elements_by_css_selector('form input')[0]
        passwordInput = self.browser.find_elements_by_css_selector('form input')[1]

        emailInput.send_keys(self.email)
        passwordInput.send_keys(self.password)
        passwordInput.send_keys(Keys.ENTER)
        time.sleep(2)
        
    def INscrape(self):
            for i in range(len(oglinks)):
                self.browser.get(oglinks[i])
                source = self.browser.page_source
                soup= BeautifulSoup(source, 'html.parser')

                descript = soup.find('script', attrs={'type': 'application/ld+json'})
                if descript == None: 
                    uploadDate.append("no info")
                else:
                    jscript = json.loads(descript.string)    
                    uploadDate.append(jscript.get("uploadDate"))
                    print(jscript.get("comment"))
                    j1 = str(jscript.get("comment"))
                    com_count.append(jscript.get("commentCount"))
                    #user_count.append(jscript.get("userInteractionCount"))  under 'interactionStatistic'
                    #cp_text.append(j1.split()[3].replace("'", '').replace(",", ''))
                    #print(jscript[comment])

                

bot = InstagramBot('529291731@qq.com', 'claytonhouse777')
bot.signIn()
bot.INscrape()

['https://www.instagram.com/p/Bz8hdNKIxJG/',
 'https://www.instagram.com/p/Bz9PsIyJGAi/',
 'https://www.instagram.com/p/Bz8eoK5HnAo/',
 'https://www.instagram.com/p/Bz8v-HUp5EM/',
 'https://www.instagram.com/p/Bz9AqRgjhnL/',
 'https://www.instagram.com/p/Bz89XnNHI82/',
 'https://www.instagram.com/p/Bz9elsinrCR/',
 'https://www.instagram.com/p/Bz9HG88JB4e/',
 'https://www.instagram.com/p/Bz89n0pFlD1/',
 'https://www.instagram.com/p/Bz9oGMupVN4/',
 'https://www.instagram.com/p/Bz9onoVBBgF/',
 'https://www.instagram.com/p/Bz9olM2B5xh/',
 'https://www.instagram.com/p/Bz9olE_JvOz/',
 'https://www.instagram.com/p/Bz9oj7Dgo4Q/',
 'https://www.instagram.com/p/Bz9ojP4B__t/',
 'https://www.instagram.com/p/Bz9oimepQay/',
 'https://www.instagram.com/p/Bz9ogu3J2OP/',
 'https://www.instagram.com/p/Bz9of9PDkMK/',
 'https://www.instagram.com/p/Bz9oeWmBzy7/',
 'https://www.instagram.com/p/Bz9od6gA72U/',
 'https://www.instagram.com/p/Bz9od4IFzx7/',
 'https://www.instagram.com/p/Bz9oE2XnHgy/',
 'https://